In [1]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [2]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [3]:
df_cq_tr = pd.read_csv(generaPath('I', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('I', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('I', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('I', 'df_cc_re.csv'))


In [4]:
df_cq_tr

,conteo,tupla,X,L,R,M,T,P,Pr,PrX,PrL,PrM
0,1,10|11|15|28|35,218,153,65,153|283,153|283,0.002,0.667,1.000,1.000,0.002
1,1,10|11|15|31|38,218,202,16,202|234,202|234,0.002,0.667,1.000,1.000,0.002
2,1,10|11|37|40|41,218,92,126,92|344,92|344,0.002,0.667,1.000,1.000,0.002
3,1,10|12|13|32|38,218,272,54,272|164,272|164,0.002,0.667,1.000,1.000,0.002
4,1,10|12|16|17|27,218,332,114,332|104,332|104,0.002,0.667,1.000,1.000,0.002
...,...,...,...,...,...,...,...,...,...,...,...,...
432,1,9|17|28|31|40,218,129,89,129|307,129|307,0.002,0.667,1.000,1.000,0.002
433,1,9|17|28|35|38,218,86,132,86|350,86|350,0.002,0.667,1.000,1.000,0.002
434,1,9|19|20|36|37,218,79,139,79|357,79|357,0.002,0.667,1.000,1.000,0.002
435,1,9|20|24|31|33,218,420,202,420|16,420|16,0.002,0.667,1.000,1.000,0.002


#### Introduzca datos para validar cuartetos y quintetos

In [5]:
def cayo_antestetos(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []  
    resta = []
    for index, row in conteos.iterrows():
        sbs = str(row[1])
        splitedtupla= unsplited_tupla(sbs)
        sbs1 = [str(splitedtupla[0])]
        sbs2 = [str(splitedtupla[1])]
        sbs3 = [str(splitedtupla[2])]
        sbs4 = [str(splitedtupla[3])]
        sbs5 = [str(splitedtupla[4])]
        dftx = df[(df.n1.isin(sbs1) | df.n2.isin(sbs1) |df.n3.isin(sbs1) | df.n4.isin(sbs1) |df.n5.isin(sbs1)) 
                & (df.n1.isin(sbs2) | df.n2.isin(sbs2) |df.n3.isin(sbs2) | df.n4.isin(sbs2) |df.n5.isin(sbs2)) 
                & (df.n1.isin(sbs3) | df.n2.isin(sbs3) |df.n3.isin(sbs3) | df.n4.isin(sbs3) |df.n5.isin(sbs3))
                & (df.n1.isin(sbs4) | df.n2.isin(sbs4) |df.n3.isin(sbs4) | df.n4.isin(sbs4) |df.n5.isin(sbs4))
                & (df.n1.isin(sbs5) | df.n2.isin(sbs5) |df.n3.isin(sbs5) | df.n4.isin(sbs5) |df.n5.isin(sbs5)) ]
        #print(dftx)
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        resta.append(abs(int(df_tupla[0])- int(calc_media(df_tupla))))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['R'] = resta
    dft['M'] = modaa
    df_filt2 = dft.iloc[0::1,[0]]
    #print("df_filt2", df_filt2)
    df_pesos = sumarcol_peso(df_filt2)
    #print("valor de df_pesos ", df_pesos)
    dft['T'] = delta
    dft['P'] = df_pesos
    proba, proba1,proba2,proba3 = how_hot_is(dft, df_tr_fu_filt_delta)
    dft['Pr']= proba
    dft['PrX'] = proba1
    dft['PrL'] = proba2
    dft['PrM'] = proba3
    return dft

In [6]:
def construir_linea(n1,n2,n3,n4,n5):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    return salida

In [7]:
def construir_cuarteto(n1,n2,n3,n4):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4)
    return salida

In [8]:
def ingresar_sorteo_validar():
    n1 = int(input(f"ingresa el primer numero del sorteo: "))
    n2 = int(input(f"ingresa el segundo numero del sorteo: "))
    n3 = int(input(f"ingresa el tercer numero del sorteo: "))
    n4 = int(input(f"ingresa el cuarto numero del sorteo: "))    
    n5 = int(input(f"ingresa el quinto numero del sorteo: "))
    quinteto = construir_linea(n1, n2, n3, n4, n5)
    cuarteto1 = construir_cuarteto(n1, n2, n3, n4)
    cuarteto2 = construir_cuarteto(n1, n2, n3, n5)
    cuarteto3 = construir_cuarteto(n1, n2, n4, n5)
    cuarteto4 = construir_cuarteto(n1, n3, n4, n5)
    cuarteto5 = construir_cuarteto(n2, n3, n4, n5)
    return quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5

In [9]:
def find_previous_coincidence(value_to_search, df):
    df_found = df.loc[df['tupla']== value_to_search]
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        return "Records Found: " + str(total_rows)
    else:
        return "No Records Found"

In [24]:
quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5 = ingresar_sorteo_validar()

ingresa el primer numero del sorteo:  5
ingresa el segundo numero del sorteo:  23
ingresa el tercer numero del sorteo:  39
ingresa el cuarto numero del sorteo:  41
ingresa el quinto numero del sorteo:  43


In [25]:
print(find_previous_coincidence(quinteto, df_cq_tr))

No Records Found


In [26]:
print(find_previous_coincidence(quinteto, df_cq_re))

No Records Found


In [27]:
print(find_previous_coincidence(cuarteto1, df_cc_tr))

No Records Found


In [28]:
print(find_previous_coincidence(cuarteto1, df_cc_re))

No Records Found


In [29]:
print(find_previous_coincidence(cuarteto2, df_cc_tr))

No Records Found


In [30]:
print(find_previous_coincidence(cuarteto2, df_cc_re))

No Records Found


In [31]:
print(find_previous_coincidence(cuarteto3, df_cc_tr))

No Records Found


In [32]:
print(find_previous_coincidence(cuarteto3, df_cc_re))

No Records Found


In [33]:
print(find_previous_coincidence(cuarteto4, df_cc_tr))

No Records Found


In [34]:
print(find_previous_coincidence(cuarteto4, df_cc_re))

No Records Found


In [35]:
print(find_previous_coincidence(cuarteto5, df_cc_tr))

No Records Found


In [36]:
print(find_previous_coincidence(cuarteto5, df_cc_re))

No Records Found
